# **1. 따릉이 API**
* https://www.bikeseoul.com/app/station/getStationRealtimeStatus.do

In [3]:
import requests
import folium
import json
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

### 1-1. 데이터 요청하기


In [5]:
targetsite = 'https://www.bikeseoul.com/app/station/getStationRealtimeStatus.do'
request = requests.post(targetsite, data={'stationGrpSeq':'ALL'})
print(request)
# print(request.text)

<Response [200]>


### 1-2. json 데이터 처리하기
* json.loads(): json 타입의 문자열 데이터를 파이썬에서 처리할 수 있도록 변환 ( 딕셔너리로 변환 )

In [6]:
bike_json = json.loads(request.text)
print(bike_json)
print(type(bike_json))

{'stationImgPath': '/nas_link/spb/attachFiles/file_admin/basePath', 'appUserSessionVO': {'lang': 'LAG_001', 'regDttm': None, 'encPwd': None, 'usrClsCd': None, 'usrDeviceId': None, 'loginId': None, 'mbId': None, 'mpnLostYn': None, 'snsType': None, 'usrSeq': None, 'voucherEndDttm': None, 'voucherSeq': None, 'usrType': None, 'usrMpnNo': None, 'appOsType': None, 'orgType': None, 'snsId': None, 'viewFlg': 'list', 'usrBirthDate': None, 'sexCd': None, 'rentEncPwd': None, 'telecomClsCd': None, 'authCiVal': None, 'authClsCd': None, 'mbTelNo': None, 'mbEmailName': None, 'mbPostNo': None, 'mbAddr1': None, 'mbAddr2': None, 'parentSexCd': None, 'parentBirthDate': None, 'parentMpnNo': None, 'emailSendAgreeYn': None, 'lastLoginDttm': None, 'mbWgt': None, 'langClsCd': None, 'leaveYn': None, 'leaveReasonCd': None, 'leaveDttm': None, 'mbInfoColecAgreeDttm': None, 'mpnLostDttm': None, 'pagingYn': None, 'usrIp': None, 'partyVoucherSeq': None, 'elecVoucherSeq': None, 'requestSeq': None, 'usrDeviceType': No

### 1-3. 딕셔너리 타입의 데이터를 데이터프레임으로 변환하기
* json_normalize(): 딕셔너리의 타입의 데이터를 판다스 데이터프레임으로 변환

In [8]:
bike_df = pd.json_normalize(bike_json, 'realtimeList')
bike_df

,stationName,stationImgFileName,stationId,stationLongitude,stationLatitude,rackTotCnt,parkingBikeTotCnt,parkingQRBikeCnt,parkingELECBikeCnt,stationSeCd,mode
0,102. 망원역 1번출구 앞,,ST-4,126.91062927,37.55564880,15,0,5,11,RAK_002,None
1,103. 망원역 2번출구 앞,,ST-5,126.91083527,37.55495071,14,0,2,0,RAK_002,None
2,104. 합정역 1번출구 앞,,ST-6,126.91508484,37.55073929,13,0,3,0,RAK_002,None
3,105. 합정역 5번출구 앞,,ST-7,126.91482544,37.55000687,5,0,0,0,RAK_002,None
4,106. 합정역 7번출구 앞,,ST-8,126.91282654,37.54864502,12,0,0,0,RAK_002,None
...,...,...,...,...,...,...,...,...,...,...,...
2730,6058. 서울도시건축전시관 옆,,ST-3297,126.97684479,37.56653976,10,0,0,0,RAK_002,None
2731,6058. 서울도시건축전시관 옆,,ST-3297,126.97684479,37.56653976,10,0,0,0,RAK_002,None
2732,6171. 월드빌딩 앞,,ST-3276,126.83743286,37.54098129,12,0,4,0,RAK_002,None
2733,6172. 가양5단지아파트,,ST-3281,126.85464478,37.56447983,10,0,12,0,RAK_002,None


In [9]:
bike_df.columns

Index(['stationName', 'stationImgFileName', 'stationId', 'stationLongitude',
       'stationLatitude', 'rackTotCnt', 'parkingBikeTotCnt',
       'parkingQRBikeCnt', 'parkingELECBikeCnt', 'stationSeCd', 'mode'],
      dtype='object')

* stationName : 대여소 이름
* stationId : 고유한 대여소 번호
* stationLongitude : 대여소 경도
* stationLatitude : 대여소 위도
* rackTotCnt : 주차 가능한 전체 자전거 대수
* parkingBikeTotCnt : 주차된 따릉이 총 대수
* parkingQRBikeCnt : 주차된 따릉이 QR형 총 대수
* parkingELECBikeCnt : 주차된 새싹 따릉이 총 대수

In [10]:
bike_df_map = bike_df[['stationName', 'stationId', 'stationLongitude', 'stationLatitude', 'rackTotCnt', 'parkingBikeTotCnt', 'parkingQRBikeCnt', 'parkingELECBikeCnt']]
bike_df_map

,stationName,stationId,stationLongitude,stationLatitude,rackTotCnt,parkingBikeTotCnt,parkingQRBikeCnt,parkingELECBikeCnt
0,102. 망원역 1번출구 앞,ST-4,126.91062927,37.55564880,15,0,5,11
1,103. 망원역 2번출구 앞,ST-5,126.91083527,37.55495071,14,0,2,0
2,104. 합정역 1번출구 앞,ST-6,126.91508484,37.55073929,13,0,3,0
3,105. 합정역 5번출구 앞,ST-7,126.91482544,37.55000687,5,0,0,0
4,106. 합정역 7번출구 앞,ST-8,126.91282654,37.54864502,12,0,0,0
...,...,...,...,...,...,...,...,...
2730,6058. 서울도시건축전시관 옆,ST-3297,126.97684479,37.56653976,10,0,0,0
2731,6058. 서울도시건축전시관 옆,ST-3297,126.97684479,37.56653976,10,0,0,0
2732,6171. 월드빌딩 앞,ST-3276,126.83743286,37.54098129,12,0,4,0
2733,6172. 가양5단지아파트,ST-3281,126.85464478,37.56447983,10,0,12,0


In [11]:
# 위도, 경도 -> float 변환

bike_df_map['stationLatitude'] = bike_df_map['stationLatitude'].astype(float)
bike_df_map['stationLongitude'] = bike_df_map['stationLongitude'].astype(float)

# 주차할 수 있는 자전거 대수, 주차된 자전거 총 대수, 주차된 QR자전거 총 대수, 주차된 새싹 자전거 총 대수 -> int

bike_df_map['rackTotCnt'] = bike_df_map['rackTotCnt'].astype(int)
bike_df_map['parkingBikeTotCnt'] = bike_df_map['parkingBikeTotCnt'].astype(int)
bike_df_map['parkingQRBikeCnt'] = bike_df_map['parkingQRBikeCnt'].astype(int)
bike_df_map['parkingELECBikeCnt'] = bike_df_map['parkingELECBikeCnt'].astype(int)

# 파생변수 만들기[total]
# 따릉이 + QR + 새싹
bike_df_map['total'] = bike_df_map['parkingBikeTotCnt'] + bike_df_map['parkingQRBikeCnt'] + bike_df_map['parkingELECBikeCnt']
bike_df_map


,stationName,stationId,stationLongitude,stationLatitude,rackTotCnt,parkingBikeTotCnt,parkingQRBikeCnt,parkingELECBikeCnt,total
0,102. 망원역 1번출구 앞,ST-4,126.910629,37.555649,15,0,5,11,16
1,103. 망원역 2번출구 앞,ST-5,126.910835,37.554951,14,0,2,0,2
2,104. 합정역 1번출구 앞,ST-6,126.915085,37.550739,13,0,3,0,3
3,105. 합정역 5번출구 앞,ST-7,126.914825,37.550007,5,0,0,0,0
4,106. 합정역 7번출구 앞,ST-8,126.912827,37.548645,12,0,0,0,0
...,...,...,...,...,...,...,...,...,...
2730,6058. 서울도시건축전시관 옆,ST-3297,126.976845,37.566540,10,0,0,0,0
2731,6058. 서울도시건축전시관 옆,ST-3297,126.976845,37.566540,10,0,0,0,0
2732,6171. 월드빌딩 앞,ST-3276,126.837433,37.540981,12,0,4,0,4
2733,6172. 가양5단지아파트,ST-3281,126.854645,37.564480,10,0,12,0,12


In [12]:
bike_df_map.dtypes

stationName            object
stationId              object
stationLongitude      float64
stationLatitude       float64
rackTotCnt              int64
parkingBikeTotCnt       int64
parkingQRBikeCnt        int64
parkingELECBikeCnt      int64
total                   int64
dtype: object

In [13]:
bike_df_map.head()

,stationName,stationId,stationLongitude,stationLatitude,rackTotCnt,parkingBikeTotCnt,parkingQRBikeCnt,parkingELECBikeCnt,total
0,102. 망원역 1번출구 앞,ST-4,126.910629,37.555649,15,0,5,11,16
1,103. 망원역 2번출구 앞,ST-5,126.910835,37.554951,14,0,2,0,2
2,104. 합정역 1번출구 앞,ST-6,126.915085,37.550739,13,0,3,0,3
3,105. 합정역 5번출구 앞,ST-7,126.914825,37.550007,5,0,0,0,0
4,106. 합정역 7번출구 앞,ST-8,126.912827,37.548645,12,0,0,0,0


In [14]:
bike_df_map.shape

(2735, 9)

In [27]:
# {정류소명} 일반:{}대, QR:{}대, 새싹:{}대, 총:{}대
# data = bike_df_map
# map = folium.Map(location=[data['stationLongitude'].mean(), data['stationLatitude'].mean()], zoom_start=15)

# for i in range(len(bike_df_map)):
#   popup = folium.Popup(str(bike_df_map.iloc[i]['stationName'])+' - '+ '\n' + '일반 : '+ str(bike_df_map.iloc[i]['parkingBikeTotCnt']) + str(bike_df_map.iloc[i]['parkingQRBikeCnt']) + str(bike_df_map.iloc[i]['parkingELECBikeCnt']), str(bike_df_map.iloc[i]['total']))
#   folium.Marker(location=[bike_df_map.iloc[i]['stationLongitude'], bike_df_map.iloc[i]['stationLatitude']], popup=popup).add_to(map)
# map

bike_map = folium.Map(location=[bike_df_map['stationLongitude'].mean(),
                      bike_df_map['stationLatitude'].mean()],
                                zoom_start=12)

for index, data in bike_df_map.iterrows():
  popup_str = '{} 일반{}대, QR:{}대, 새싹:{}대, 총:{}대'.format(data['stationName'], data['parkingBikeTotCnt'], data['parkingQRBikeCnt'], data['parkingELECBikeCnt'], data['total'])
  popup = folium.Popup(popup_str, max_width=600)
  folium.Marker(location=[data['stationLongitude'], data['stationLatitude']], popup=popup).add_to(bike_map)

bike_map

Output hidden; open in https://colab.research.google.com to view.